# Tool-Calling Fine-Tuning with SFT

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ProfSynapse/Toolset-Training/blob/main/Trainers/notebooks/sft_colab_tool_calling.ipynb)

## 🎓 What You'll Learn

This notebook teaches you how to fine-tune a language model to use the **Claudesidian-MCP toolset** for Obsidian vault operations. By the end, you'll have:

- **A custom AI model** that can call tools to manage Obsidian vaults
- **Hands-on experience** with supervised fine-tuning (SFT)
- **Understanding** of hyperparameters and how they affect training

## 🔬 What is SFT?

**SFT (Supervised Fine-Tuning)** is like teaching through examples:
- You show the model examples of correct tool-calling behavior
- The model learns to replicate those patterns
- Use SFT when teaching a model a **new skill** (like using tools)

**When to use SFT:**
- ✅ Teaching tool-calling from scratch
- ✅ Learning new task formats
- ✅ Initial training with positive examples

**Not for:**
- ❌ Refining existing behavior (use KTO instead)
- ❌ Teaching preferences between good/bad outputs (use preference learning)

## 💻 Hardware Requirements

**Recommended GPU:**
- 7B models: T4 (15GB VRAM) - ✅ **Free Colab tier works!**
- 13B models: A100 (40GB VRAM) - Colab Pro
- 70B models: A100 (80GB VRAM) - Colab Pro+

**Training time:** ~45 minutes for a 7B model

## 1. Installation

Install Unsloth and dependencies. This takes ~2 minutes.

In [ ]:
# Install Unsloth for faster training
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# Install training dependencies
%%capture
!pip install -U "transformers>=4.45.0"
!pip install "datasets==4.3.0"
!pip install -U accelerate bitsandbytes
!pip install -U trl peft xformers triton

## 2. Mount Google Drive (Optional)

Save checkpoints to Google Drive so they persist if runtime disconnects.

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# Create output directory
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/SFT_Training"
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)

print(f"✓ Google Drive mounted")
print(f"✓ Checkpoints will be saved to: {DRIVE_OUTPUT_DIR}")

## 3. HuggingFace Credentials

Add your HF token to Colab secrets:
1. Click the 🔑 key icon in the left sidebar
2. Add new secret: `HF_TOKEN`
3. Get token from https://huggingface.co/settings/tokens

In [ ]:
import os
from google.colab import userdata
from huggingface_hub import HfApi

# Get token from Colab secrets
HF_TOKEN = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = HF_TOKEN

# Get your HuggingFace username automatically
api = HfApi()
hf_user = api.whoami(token=HF_TOKEN)["name"]

print(f"✓ HuggingFace token loaded")
print(f"✓ Username: {hf_user}")

## 4. Model Configuration

**What this does:** Choose the base model you want to fine-tune and configure basic settings.

Think of this like choosing which "brain" you want to teach tool-calling skills to.

In [ ]:
# @title ⚙️ Model & Dataset Configuration
# @markdown Use the dropdowns to select your model and configure your dataset.

# @markdown ### 🧠 Base Model Selection
# @markdown Choose a model based on your VRAM availability. Models are ordered by size (1B - 24B).
# @markdown * **1B-3B:** Fast, runs on any GPU
# @markdown * **7B-9B:** Standard balance of speed/intelligence
# @markdown * **12B-24B:** High intelligence, requires ~12GB-24GB VRAM (A100 recommended for 20B+)
MODEL_NAME = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit" # @param ["unsloth/Llama-3.2-1B-Instruct-bnb-4bit", "unsloth/Qwen2.5-1.5B-Instruct-bnb-4bit", "unsloth/gemma-2-2b-it-bnb-4bit", "unsloth/Llama-3.2-3B-Instruct-bnb-4bit", "unsloth/Qwen2.5-3B-Instruct-bnb-4bit", "unsloth/Phi-3.5-mini-instruct", "unsloth/Qwen2.5-7B-Instruct-bnb-4bit", "unsloth/mistral-7b-v0.3-bnb-4bit", "unsloth/mistral-7b-instruct-v0.3-bnb-4bit", "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",  "unsloth/DeepSeek-R1-0528-Qwen3-8B-unsloth-bnb-4bit", "unsloth/gemma-2-9b-it-bnb-4bit", "unsloth/Llama-3.2-11B-Vision-Instruct-unsloth-bnb-4bit", "unsloth/Mistral-Nemo-Instruct-v1-bnb-4bit", "unsloth/gemma-3-12b-it-unsloth-bnb-4bit", "unsloth/Qwen2.5-14B-Instruct-bnb-4bit", "unsloth/Phi-4", "unsloth/gpt-oss-20b-unsloth-bnb-4bit", "unsloth/Mistral-Small-24B-Instruct-2501-bnb-4bit", "unsloth/Magistral-Small-2509-unsloth-bnb-4bit"]

# @markdown ### 📏 Max Output Length
# @markdown  2048 is standard. Higher values require more VRAM.
MAX_SEQ_LENGTH = 2048 # @param [1024, 2048, 4096, 8192] {type:"raw"}

# @markdown ### 📚 Dataset Configuration
DATASET_NAME = "professorsynapse/claudesidian-synthetic-dataset" # @param {type:"string"}
DATASET_FILE = "syngen_tools_sft_11.23.25_toolcall.jsonl" # @param {type:"string"}

# @markdown ### 🏷️ Output Model Name
# @markdown Name your fine-tuned model (e.g., `my-tool-model-v1`).
OUTPUT_MODEL_NAME = "nexus-tools-sft-7b" # @param {type:"string"}

print(f"✓ Configuration set:")
print(f"  • Model: {MODEL_NAME}")
print(f"  • Context: {MAX_SEQ_LENGTH}")
print(f"  • Dataset: {DATASET_NAME}/{DATASET_FILE}")
print(f"  • Output: {OUTPUT_MODEL_NAME}")

## 5. Load Model and Tokenizer

**What this does:** Downloads the base model and prepares it for training.

The model is the "brain" that will learn tool-calling. The tokenizer converts text into numbers the model can process. We use 4-bit quantization to fit large models into limited GPU memory.

In [ ]:
from unsloth import FastLanguageModel
import torch

# Check GPU
print(f"Using GPU: {torch.cuda.get_device_name(0)}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Available VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print()

In [ ]:
# Load the base model and tokenizer from HuggingFace
# This downloads the model weights (~7GB for 7B models)
#
# Parameters explained:
#   model_name: Which model to download
#   max_seq_length: Max tokens model can process at once
#   dtype=None: Auto-detect best precision for your GPU
#   load_in_4bit=True: Use 4-bit quantization to save memory
#   token: Your HF token for accessing the model

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect (usually bfloat16 or float16)
    load_in_4bit=True,  # Reduces memory usage by ~75%
    token=HF_TOKEN,
)

print("✓ Model loaded successfully")
print(f"  Model has {sum(p.numel() for p in model.parameters()):,} parameters")

## 6. Apply LoRA Adapters

**What this does:** Add trainable "adapter" layers to the model instead of training the entire thing.

Think of LoRA like teaching a new skill through muscle memory - we add small specialized layers that learn the new behavior, while keeping the main "brain" frozen. This is way faster and uses less memory than retraining everything.

In [ ]:
# @title 🔧 LoRA Adapter Configuration
# @markdown Configure the size and strength of the fine-tuning adapters.

# @markdown ### 🎛️ LoRA Parameters
# @markdown **Rank (r):** Higher = smarter but slower/more memory (Standard: 16-64).
# @markdown Alpha will be automatically set to 2 * r.
r = 32 # @param [8, 16, 32, 64, 128] {type:"raw"}

lora_alpha = r * 2

# @markdown **Dropout:** Helps prevent overfitting (Standard: 0.05).
lora_dropout = 0.05 # @param {type:"number"}

# @markdown **Random Seed:** Change this for different initialization.
random_state = 3407 # @param {type:"integer"}

model = FastLanguageModel.get_peft_model(
    model,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    use_gradient_checkpointing="unsloth",
    random_state=random_state,
)

print(f"✓ LoRA adapters applied:")
print(f"  • Rank: {r}")
print(f"  • Alpha: {lora_alpha}")
print(f"  • Dropout: {lora_dropout}")
print(f"  • Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

## 7. Load and Prepare Dataset

**What this does:** Downloads training examples and formats them for the model.

The dataset contains ~5,500 examples of correct tool-calling behavior. Think of it like a textbook full of solved problems that the model will learn from.

In [ ]:
from datasets import load_dataset

"""
═══════════════════════════════════════════════════════════════════════════
LOAD DATASET FROM HUGGINGFACE
═══════════════════════════════════════════════════════════════════════════
This downloads pre-made training examples of correct tool-calling behavior.
Each example shows: user request → tool call → result → assistant response
"""

print(f"Loading dataset: {DATASET_NAME}")
dataset = load_dataset(
    DATASET_NAME,  # HuggingFace repository containing the dataset
    data_files=DATASET_FILE,  # Specific JSONL file to use
    split="train"  # Use the training split
)

print(f"✓ Loaded {len(dataset)} examples")
print(f"\nSample example:")
print(dataset[0])

"""
═══════════════════════════════════════════════════════════════════════════
SET CHAT TEMPLATE (Model-Specific)
═══════════════════════════════════════════════════════════════════════════
The chat template defines how conversations are formatted for the model.
Different models use different formats - we auto-detect and use the right one!
"""

# Mistral-specific template (uses [INST] format)
MISTRAL_CHAT_TEMPLATE = """{{ bos_token }}{% for message in messages %}{% if message['role'] == 'system' %}{% if loop.index == 1 %}{{ message['content'] + ' ' }}{% endif %}{% elif message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ ' ' + message['content'] + eos_token }}{% endif %}{% endfor %}"""

# Generic fallback template for other models
DEFAULT_CHAT_TEMPLATE = """{% for message in messages %}{% if message['role'] == 'user' %}{{ '<|user|>\\n' + message['content'] + eos_token }}{% elif message['role'] == 'system' %}{{ '<|system|>\\n' + message['content'] + eos_token }}{% elif message['role'] == 'assistant' %}{{ '<|assistant|>\\n'  + message['content'] + eos_token }}{% endif %}{% if loop.last and add_generation_prompt %}{{ '<|assistant|>' }}{% endif %}{% endfor %}"""

if tokenizer.chat_template is None:
    # Detect model type and use appropriate template
    is_mistral = 'mistral' in MODEL_NAME.lower()

    if is_mistral:
        print("\n✓ Detected Mistral model - using [INST] format")
        tokenizer.chat_template = MISTRAL_CHAT_TEMPLATE
        print("   Format: <s>[INST] user [/INST] assistant</s>")
        print("   This is the official Mistral chat format!")
    else:
        print("\n⚠️  No chat template found, using default format")
        tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE
        print("   Format: <|user|>\\ncontent</s>")
else:
    print("\n✓ Tokenizer already has chat template")
    if 'mistral' in MODEL_NAME.lower():
        print("   Using Mistral model - template should use [INST] format")

In [ ]:
"""
═══════════════════════════════════════════════════════════════════════════
FORMAT DATASET FOR TRAINING
═══════════════════════════════════════════════════════════════════════════
Convert the conversation format into the exact text format the model expects.
This applies the chat template to each example.
"""

def format_chat_template(example):
    """
    Convert conversations to tokenizer's chat template.

    Input: {"conversations": [{"role": "user", "content": "..."}, ...]}
    Output: {"text": "<|im_start|>user\n...<|im_end|>\n..."}
    """
    text = tokenizer.apply_chat_template(
        example["conversations"],  # List of message dicts
        tokenize=False,  # Return string, not token IDs
        add_generation_prompt=False  # Don't add prompt for model to continue
    )
    return {"text": text}

# Apply formatting to entire dataset
# This creates a new "text" field with formatted conversations
dataset = dataset.map(
    format_chat_template,  # Function to apply
    remove_columns=dataset.column_names,  # Remove original columns
    desc="Formatting dataset"  # Progress bar description
)

print("✓ Dataset formatted for training")
print(f"\nFormatted example (first 500 characters):")
print(dataset[0]["text"][:500])
print("\n💡 The full formatted text will be used during training")

## 8. Training Configuration

**What this does:** Set the hyperparameters that control how the model learns.

This is the **most important section** - these settings determine how fast the model learns, how much memory it uses, and how good the final result will be. Think of it like configuring a study plan: how many hours per day, how many review sessions, how to handle difficult material, etc.

In [ ]:
from trl import SFTTrainer, SFTConfig
from unsloth import is_bfloat16_supported
from datetime import datetime

# Create timestamped output directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"{DRIVE_OUTPUT_DIR}/{timestamp}"

# @title 🏃 Training Hyperparameters
# @markdown Control the speed and quality of training.

# @markdown ### ⚡ Performance Settings
# @markdown **Batch Size:** Examples per step. Lower if you run out of memory.
per_device_train_batch_size = 2 # @param [1, 2, 4, 6, 8, 10, 12] {type:"raw"}

# @markdown **Gradient Accumulation:** Simulates larger batches.
gradient_accumulation_steps = 4 # @param [1, 2, 4, 6, 8, 10, 12] {type:"raw"}

# @markdown ### 🧠 Learning Rate Configuration
# @markdown **Step 1: Choose the Magnitude (Exponent)**
# @markdown This is the most important setting. It determines the "speed" of learning.
# @markdown * **4** = Standard (1e-4). Recommended for 7B models and SFT.
# @markdown * **5** = Slow (1e-5). Use if training is unstable or for larger models.
# @markdown * **6** = Very Slow (1e-6). Precise but takes much longer.
learning_rate_exponent = 4 # @param [4, 5, 6, 7] {type:"raw"}

# @markdown **Step 2: Choose the Multiplier**
# @markdown Fine-tunes the rate within that magnitude (e.g., Multiplier 2 + Exponent 4 = 2e-4).
learning_rate_multiplier = 2 # @param [1, 2, 3, 4, 5, 6, 7, 8, 9] {type:"raw"}

learning_rate = learning_rate_multiplier * (10 ** -learning_rate_exponent)

# @markdown ### 🔄 Epochs
# @markdown Number of passes through the dataset.
num_train_epochs = 3 # @param {type:"integer"}

# @markdown ### 💾 Saving & Logging
save_steps = 100 # @param {type:"integer"}
logging_steps = 5 # @param {type:"integer"}

training_args = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    max_grad_norm=1.0,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    num_train_epochs=num_train_epochs,
    max_seq_length=MAX_SEQ_LENGTH,
    packing=False,
    dataset_text_field="text",
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    optim="adamw_8bit",
    gradient_checkpointing=True,
    logging_steps=logging_steps,
    save_steps=save_steps,
    save_total_limit=3,
    seed=42,
    report_to="none",
)

print("✓ Training configuration ready")
print(f"  • Batch Size: {per_device_train_batch_size}")
print(f"  • Gradient Accum.: {gradient_accumulation_steps}")
print(f"  • Learning Rate: {learning_rate} ({learning_rate_multiplier}e-{learning_rate_exponent})")
print(f"  • Epochs: {num_train_epochs}")
print(f"  • Output Dir: {output_dir}")

## 9. Initialize Trainer

**What this does:** Creates the training engine that coordinates everything.

The SFTTrainer is the orchestrator - it takes the model, dataset, and configuration, then handles all the training mechanics (gradient updates, checkpointing, logging, etc.).

In [ ]:
# Create the SFTTrainer
# This combines the model, dataset, and configuration into one training pipeline

trainer = SFTTrainer(
    model=model,  # The model with LoRA adapters
    tokenizer=tokenizer,  # For converting text to tokens
    train_dataset=dataset,  # Our formatted training examples
    args=training_args,  # All the hyperparameters we configured
)

print("✓ Trainer initialized")
print("  Ready to start training!")

## 10. Train!

**What this does:** The actual learning happens here!

The model will:
1. **Read examples** from the dataset
2. **Predict** what the response should be
3. **Compare** its prediction to the correct answer
4. **Update weights** to get closer to the correct answer
5. **Repeat** this process for 3 epochs (3 full passes through the data)

**What to expect:**
- Training takes ~45 minutes for 7B models on T4 GPU
- You'll see progress updates every 5 steps
- Loss should generally decrease over time (learning is working!)
- Checkpoints are saved every 100 steps to Google Drive

**What the metrics mean:**
- **Loss:** How "wrong" the model is (lower = better, aim for <1.0)
- **Learning Rate:** Gradually decreases as training progresses
- **Samples/sec:** Training speed (depends on GPU)

**💾 Checkpoint Resumption:**
If your Colab session disconnects, don't worry! Your checkpoints are saved to Google Drive. You can resume training by:
1. Re-running cells 1-9 (setup, model loading, dataset prep, config)
2. In the training cell below, the code will automatically detect the latest checkpoint and resume from there
3. Your progress is preserved!

In [ ]:
# Check GPU memory
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")
print()

# ════════════════════════════════════════════════════════════════════════════
# 🔍 Check for existing checkpoints (automatic resumption)
# ════════════════════════════════════════════════════════════════════════════
import glob
import os

checkpoint_dirs = sorted(glob.glob(f"{output_dir}/checkpoint-*"))
resume_from_checkpoint = None

if checkpoint_dirs:
    # Found checkpoints - get the latest one
    latest_checkpoint = max(checkpoint_dirs, key=lambda x: int(x.split("-")[-1]))
    resume_from_checkpoint = latest_checkpoint
    print(f"✓ Found existing checkpoint: {os.path.basename(latest_checkpoint)}")
    print(f"  Resuming training from this checkpoint")
    print(f"  Total checkpoints found: {len(checkpoint_dirs)}")
else:
    print("ℹ️  No existing checkpoints found - starting fresh training")

print()

# Start training (or resume)
print("=" * 60)
if resume_from_checkpoint:
    print("RESUMING TRAINING FROM CHECKPOINT")
else:
    print("STARTING TRAINING")
print("=" * 60)
print()

trainer_stats = trainer.train(resume_from_checkpoint=resume_from_checkpoint)

print()
print("=" * 60)
print("TRAINING COMPLETED")
print("=" * 60)

## 11. Upload to HuggingFace

**What this does:** Share your trained model with the world!

We'll create **three versions** of your model:

1. **LoRA adapters** - Small files that contain just the changes
   - Fast to download
   - Need to be combined with base model to use
   
2. **Merged 16-bit model** - Full model with adapters merged in
   - High quality, no precision loss
   - Large file size
   - Best for local deployment (Ollama, LM Studio)
   
3. **GGUF quantizations** - Optimized versions for CPU/GPU inference
   - Q4_K_M - Recommended for most users
   - Q5_K_M - Better quality, larger size
   - Q8_0 - Nearly full quality
   - These work directly with Ollama and LM Studio!

In [ ]:
# Upload LoRA adapters
model.push_to_hub(
    f"{hf_user}/{OUTPUT_MODEL_NAME}",
    token=HF_TOKEN,
    private=False
)
tokenizer.push_to_hub(
    f"{hf_user}/{OUTPUT_MODEL_NAME}",
    token=HF_TOKEN,
    private=False
)

print(f"✓ LoRA adapters uploaded to HuggingFace")
print(f"  View at: https://huggingface.co/{hf_user}/{OUTPUT_MODEL_NAME}")

In [ ]:
# Upload merged 16-bit model
print("Merging LoRA weights into base model (16-bit)...")
print("This will take ~5 minutes...")

model.push_to_hub_merged(
    f"{hf_user}/{OUTPUT_MODEL_NAME}-merged",
    tokenizer,
    save_method="merged_16bit",
    token=HF_TOKEN,
    private=False
)

print(f"✓ Merged model uploaded to HuggingFace")
print(f"  View at: https://huggingface.co/{hf_user}/{OUTPUT_MODEL_NAME}-merged")

In [ ]:
# Create GGUF quantizations
quantization_methods = ["q4_k_m", "q5_k_m", "q8_0"]

print("Creating GGUF quantizations...")
print(f"This will create {len(quantization_methods)} versions")
print()

model.push_to_hub_gguf(
    f"{hf_user}/{OUTPUT_MODEL_NAME}",
    tokenizer,
    quantization_method=quantization_methods,
    token=HF_TOKEN,
)

print()
print("✓ GGUF quantizations created and uploaded!")
print(f"  View at: https://huggingface.co/{hf_user}/{OUTPUT_MODEL_NAME}")

## 🎉 Done!

Your model has been trained and uploaded to HuggingFace!

## 📊 What You Accomplished

✅ **Fine-tuned a language model** to use the Claudesidian toolset  
✅ **Learned about SFT hyperparameters** and how they affect training  
✅ **Created multiple model formats** (LoRA, merged, GGUF) for different use cases  
✅ **Published your model** to HuggingFace for others to use  

## 🚀 Next Steps

### 1. Test Your Model

**Using LM Studio:**
1. Open LM Studio → "Discover" tab
2. Search for your username or. model name
3. Download the GGUF version of your choice
4. Load and test with tool-calling prompts!

**Using Ollama:**
```bash
# Download your model
ollama pull {hf_user}/{OUTPUT_MODEL_NAME}

# Test it
ollama run {hf_user}/{OUTPUT_MODEL_NAME}
```

### 2. Evaluate Quality

Run the Evaluator to test tool-calling accuracy:
```bash
python Evaluator
```

---

**Questions or issues?** Check the documentation or open an issue on GitHub!